# 🔀 Dependency Inversion Principle

- https://en.wikipedia.org/wiki/Dependency_inversion_principle
    - High-level modules should not import anything from low-level modules. Both should depend on abstractions (e.g., interfaces)
    - Abstractions should not depend on details. Details (concrete implementations) should depend on abstractions

# 🔀 Inversion of Control

- https://en.wikipedia.org/wiki/Inversion_of_control
- Callback, Schedular, Event Loops

In [ ]:
// we are user
var framework = new CoolFramework()
{
    Configuration: new
    {
    },
    OnInitialization: builder =>
    {
    }
};

framework.OnLoad += (s, e) => { /* you do something */ };
framework.Step1 += (s, e) => { };       // hooks
framework.OnFinish += (s, e) => { };
framework.Initialize();
framework.Start();

In [ ]:
// we are framework
class MsiAssertions
{
    public EventHandler<..> OnMsiLoaded;
    public EventHandler<..> OnDirectoryResolved;
    public EventHandler<..> OnFilesResolved;

    public EventHandler<..> OnDeleting;
    public EventHandler<..> OnDeleted; // OnFinished

    void Load(string file) { }

    void Resolve() { }
}

# 🔀 Dependency Injection

- https://en.wikipedia.org/wiki/Dependency_injection
- Abstraction / Interfaces
- Injection
    - Constructor
    - Method Injection and Setter Injection
    - Interface Injection

In [ ]:
class FromServicesAttribute : Attribute { }
interface IService {}

// Method injection
void ControllerAction(
    [FromServices]
    IService service)
{
    //service.Execute();
}

In [ ]:
// Interface injection
interface IWeapn { void Use();}
interface IWeaponInjectable { void SetWeapen(IWeapn weapon); }

// class Ken : IWeaponInjectable { }  // Street Fighter

class Match
{
    public void EquipLeftPlayer(IWeaponInjectable player, IWeapn weapen) { }
    public void EquipRightPlayer(IWeaponInjectable player, IWeapn weapen) { }
    public void Start(int round) { }
}

- Principals
    - DIP and IoC
- Design Pattern
    - DI
- Framework
    - IoC Containers

- The SOLID principals

# ➕ Entity Fields

In [ ]:
Account Number: 1000
Account Title: Khurram              Temporal (additional info, change date, previous title)
Account Balance: 600                Calculated


Account Statement

1/3     1000
5/3     -500
10/3    -300
1/4     1000
5/4     -500
10/4    -300
1/5     1000
5/5     -500
10/5    -300

# 🚀 Domain Driven Design - But Wait

<img src=images/evan-1-bounded-context.png>

<img src=images/evan-2-context-mapping.png>

<img src=images/evan-3-architecture.png>

<img src=images/evan-4-aggregates.png>

<img src=images/evan-5-domain-events.png>

# ⚡ Mediator

## Implementation using Dependency Injection

In [ ]:
#r "nuget: Microsoft.Extensions.DependencyInjection"

In [ ]:
using System.Reflection;
using System.Threading;
using Microsoft.Extensions.DependencyInjection;

interface IRequest<TResponse> // marker interface holding the response type
{ }

interface IRequestHandler<TRequest, TResponse> where TRequest : IRequest<TResponse>
{
    Task<TResponse> HandleAsync(TRequest request, CancellationToken token); // asynchronously process request giving a response
}

interface ISender
{
    // it has request handler signature; but from implementation its expected it will magically "figure out" the request handler
    Task<TResponse> SendAsync<TResponse>(IRequest<TResponse> request, CancellationToken token = default);
}

class Sender(IServiceProvider provider) : ISender
{
    // we could either have mechanism here to register handlers or use DI container to resolve them
    public Task<TResponse> SendAsync<TResponse>(IRequest<TResponse> request, CancellationToken token = default)
    {   // we are using DI container to resolve them
        var handlerType = typeof(IRequestHandler<,>).MakeGenericType(request.GetType(), typeof(TResponse));
        dynamic handler = provider.GetService(handlerType);

        return handler.Handle((dynamic)request, token);
    }
}

static IServiceCollection AddMediator(this IServiceCollection services, Assembly assembly = null)
{
    assembly ??= Assembly.GetCallingAssembly();

    var handlerTypeInterface = typeof(IRequestHandler<,>);
    var handleTypes = assembly.GetTypes()
        .Where(type => !type.IsAbstract && !type.IsInterface)
        .SelectMany(type => type.GetInterfaces()
            .Where(i => i.IsGenericType && i.GetGenericTypeDefinition() == handlerTypeInterface)
            .Select(i => new { Interface = i, Implementation = type }));
    
    foreach(var handleType in handleTypes)
        services.AddScoped(handleType.Interface, handleType.Implementation);
    
    return services;
}

## Vertical Slice Architecture

In [ ]:
static class GetTeams
{
    public record Query : IRequest<List<string>>;

    public class Handler : IRequestHandler<Query, List<string>>
    {
        public Task<List<string>> HandleAsync(Query request, CancellationToken cancellationToken)
        {
            var teams = new List<string>
            {
                "Flooid", "Juriba", "TAAS", "Khoji", "Admin"
            };

            return Task.FromResult(teams);
        }
    }
}

In [ ]:
static class MoveResource
{
    public record Command(string ResourceId, string FromTeam, string ToTeam) : IRequest<bool>;

    public class Handler : IRequestHandler<Command, bool>
    {
        public Task<bool> HandleAsync(Command command, CancellationToken cancellationToken)
        {   // Simulate resource move logic
            var moved = !string.IsNullOrWhiteSpace(command.ResourceId) &&
                !string.Equals(command.FromTeam, command.ToTeam, StringComparison.OrdinalIgnoreCase);

            return Task.FromResult(moved);
        }
    }
}

In [ ]:
[ApiController]
[Route("api/[controller]")]
public class TeamsController(ISender sender) : ControllerBase
{
    [HttpGet]
    public async Task<IActionResult> GetTeams(CancellationToken cancellationToken)
    {
        var teams = await sender.SendAsync(new GetTeams.Query(), cancellationToken);
        return Ok(teams);
    }

    [HttpPost("{resourceId}/move")]
    public async Task<IActionResult> MoveResource(string resourceId, [FromBody] MoveResource.Command command, CancellationToken cancellationToken)
    {
        if (resourceId != command.ResourceId) return BadRequest("Resource ID mismatch.");

        var moved = await sender.SendAsync(command, cancellationToken);
        return moved ? Ok() : NotFound("Resource not found or already in the target team.");
    }
}